# モンテカルロ法
方策勾配法(2_3)や方策反復法, 価値反復法(2_4_2) では遷移確率 $P$ が既知であることを前提としていた。

確かに迷路ゲームでは, 前の状態と行動が決まれば次の状態が一意に決まるためこの手法で解けた。

今回は, 遷移確率 $P$ がわからない例として, ブラックジャックゲームの学習を行う. 

## 1. ブラックジャックのルール
ブラックシャックはカジノで定番のゲームで、以下のようなルール。

1. ルールの概要
  * トランプを使用する。
  * トランプは無限デッキあると仮定する。（＝カードの出る確率は変化しない）
  * Aは1もしくは11として扱う。
  * 2〜10は数字通り扱う。
  * J, Q, Kは10として扱う。
  * カードの合計が21を越えず、出来るだけ21に近い方が勝ち。（同じなら引き分け）
  
2. プレイの流れ
  * ユーザーにカードが2枚オープンで配られる。
  * ディーラーにカードが1枚はオープン、もう1枚はクローズで配られる。
  * プレイヤーは以下の行動が出来る。
  * ヒット（カードをもう1枚引く）
  * スタンド（カードを引くのを止める）
  * カードの合計が21を越えたら、その時点でプレイヤーの負け。
  * スタンドするか21を越えるまでは、何度でもヒット出来る。
  *  プレイヤーがスタンドを選択したら、ディーラーは伏せていたカードをオープンにし、カードの合計が17以上になるまでカードを引く。
  * カードの合計が21を越えたら、その時点でプレイヤーの勝ち。
  * ディーラーのカードの合計が21以下の場合、カードの合計を比べる。
  * カードの合計が21に近い方の勝ち。同じなら引き分け。

## 2. マルコフ決定過程としてのブラックジャックゲーム
1. 状態集合 $S$ <br>
     $$ S = \{ \text{プレイヤーの状態} \}  \times \{ \text{ディーラーの状態}\} \cup \{ \text{Player勝ち}, \text{Player負け}, \text{引き分け} \}$$
     
     * プレイヤーの状態 <br>
         得点が11以下ならhitしても得点が21を超えない.  <br>
         よって, プレイヤーの手持ちの得点は12以上の状態から始まるものとできる.  <br>
         'A'は1と扱うこともできるし11と扱うこともできる. そこで, 11として扱う'A'が存在するかどうかで場合分け.
         * ( 得点合計 12 ~ 21, 11点として扱う'A'が存在 )
         * ( 得点合計 12 ~ 21, 11点として扱う'A'が存在しない )
          
     * ディーラーの状態 <br>
       　　 1つのカードは見えていない. もう1つのカードは見えていて, [ 'A', 2 ~ 10 ] の場合がある.

2. 行動集合 $A$ <br>
    $$ A = \{ \text{hit}, \text{stand} \} $$
    
3. 報酬関数 $r$ <br>
    $$ r(s, a, \text{Player勝ち} ) = 1,\ r(s, a, \text{Player負け} ) = -1,\ r(s, a, \text{引き分け} ) = 0 \ ( \forall s \in S,\ \forall a \in A)$$
    
4. 遷移関数 $P$ <br>
    hitかstandか行動を取った後にどの状態に行くかは確率的に変化する. この確率が未知とする.
    
5. 割引率 $ \gamma $ <br>
    $$\gamma = 1.0 $$


In [1]:
# ブラックジャックゲームで遊ぶ.
from blackjack import *
game = BlackJack()
while True:
    game.show_status()
    s = input("[h] : hit, [s] : stand \n")
    if s == "h":
        game.player_hit()
    elif s == "s":
        game.player_stand()
    else:
        print("Wrong Input")
        continue
    if game.finish:
        break

game.show_status()
result = game.result()
if result > 0:
    print("Player Win!")
elif result < 0:
    print("Player Lose!")
else:
    print("Draw.")

--------------------
Player (total : 15) [ 6, 9 ]
Dealer (total : ??) [ J, A, ?? ]
--------------------
[h] : hit, [s] : stand 
h
--------------------
Player (total : 24) [ 6, 9, 9 ]
Dealer (total : 21) [ J, A ]
--------------------
Player Lose!


## 3. モンテカルロ法の概要
### 3.1 方策評価
方策反復法や価値反復法では, 状態価値関数 $V^{\pi}$ を求めることで方策 $\pi$ の評価を行なっていた.
    
今回も同様に $V^{\pi},\ Q^{\pi}$ を求めたい. しかし, そもそもベルマン方程式が連立方程式として解けない.

さらに $P$ がわからない時 $Q^{\pi}$ から $V^{\pi}$ は求まるが, $V^{\pi}$ から $Q^{\pi}$ は求まらない. (参照 : 2_4_2 の 4. )

なので, $Q^{\pi}$ を定義に戻って求めたいと考える.
$$ Q^{\pi} (s,a) = \mathbb{E}[ \sum_{n=t}^{\infty} \gamma^{n-t} r_n  ; s_t = s,\ a_t = a ] $$

期待値は「理論上の平均」であることから, シミュレーションをして得られた収益の平均で期待値を推定する.
$$ \mathbb{E}[ \sum_{n=t}^{\infty} \gamma^{n-t} r_n  ; s_t = s,\ a_t = a ] \simeq \dfrac{1}{M} \sum_{m=1}^M \sum_{n=t}^{\infty} \gamma^{n-t} r_{m,n}  ; s_{m, t} = s,\ a_{m,t} = a $$

どのサンプルを採用するかによって2つの手法がある.
* 初回訪問 : 
     
* 逐次 : 

### 3.2 方策改善
方策改善は, 2_4_2 と同じように greedy または $\epsilon$-greedy で更新を行う. 再記述すると, 
* greedy : 
    $$ \pi'(s, a) \leftarrow \begin{cases} 1 \quad ( a = \underset{a'\in A}{\operatorname{argmax}} Q^{\pi}(s, a') ) \\ 0 \quad ( otherwise ) \end{cases} $$

* $\epsilon$-greedy : 
    $$ \pi'(s, a) \leftarrow \begin{cases} 1 - (|A| - 1) \epsilon \quad &( a = \underset{a'\in A}{\operatorname{argmax}} Q^{\pi}(s, a') ) \\ \epsilon & \quad ( otherwise ) \end{cases} $$
 
### 3.3 乱数シュミレーションの問題点
シュミレーションする時, 全ての状態が観測され, 各状態から様々な行動を取った結果の収益を平均化したい.
しかし, 特にgreedy方策のような決定論的な方策を取っていると1つの状態からは同じ行動しか取らず, 学習がうまくいかない。

これを解決する方法がいくつか考えられている.
* 開始点探査の仮定をおき, 任意の状態行動対を開始点としてシミュレーションする. &rarr; モンテカルロES法
開始点探査の仮定とは, 「任意の状態行動対 $(s, a) $ が出現する確率が0ではない」という仮定.
これならば方策が決定論的であってもうまくいくはず.

* 方策を決定論的でないソフトな方策 $\epsilon$-greedyに変更する. &rarr; 方策オン型モンテカルロ法

* シュミレーション時の方策( 挙動方策 ) と本番用の方策 ( 推定方策 ) を別にする. &rarr; 方策オフ型モンテカルロ法


### 3.4 モンテカルロ-ES法
ここでは, 方策評価は初回訪問の方式で行う. <br>
価値反復法のように $Q^{\pi}$ の更新と $\pi$ の更新を交互に行う.

<img src="https://yoheitaonishi.com/wp-content/uploads/2018/10/es.png" width=500 >

In [16]:
#%load_ext autoreload
#from blackjack import *

import numpy as np


class BlackJack:
    def __init__(self, s_0=None):
        self.CARD_VALUE = {'A': 11, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6,
                           '7': 7, '8': 8, '9': 9, '10': 10, 'J': 10, 'Q': 10, 'K': 10}
        self.player_cards = []
        self.dealer_cards = []
        if (s_0 is None):
            self.player_ace = 0
            self.player_total = 0

            self.dealer_ace = 0
            self.dealer_total = 0

            while self.player_total < 12:
                self.player_draw()
            for i in range(2):
                self.dealer_draw()
        else:
            self.player_total, self.player_ace, self.dealer_total = s_0
            self.dealer_ace = 0
            if self.dealer_total == 1:
                self.dealer_ace += 1
                self.dealer_total = 11
            self.dealer_draw()

        self.finish = False

    def show_status(self):
        print("-"*20)
        print("Player (total : {0}) [ {1} ]".format(
            self.player_total, ", ".join(self.player_cards)))

        if self.finish:
            print("Dealer (total : {0}) [ {1} ]".format(
                self.dealer_total, ", ".join(self.dealer_cards)))
        else:
            print("Dealer (total : ??) [ {1}, ?? ]".format(
                self.dealer_total, ", ".join(self.dealer_cards), (self.dealer_total if self.dealer_total != 11 else 1)))
        print("-"*20)

    def state(self):
        if self.finish:
            res = self.result()
            if res > 0:
                return "PlayerWin"
            elif res < 0:
                return "PlayerLose"
            else:
                return "Draw"
        else:
            return (self.player_total, int(self.player_ace > 0), self.CARD_VALUE[self.dealer_cards[0]])

    def player_hit(self):
        self.player_draw()
        if self.player_total > 21:
            self.finish = True

    def player_stand(self):
        while self.dealer_total < 17:
            self.dealer_draw()
        self.finish = True

    def player_draw(self):
        draw_card = np.random.choice(
            ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K'])
        self.player_cards.append(draw_card)
        self.player_total += self.CARD_VALUE[draw_card]
        if draw_card == 'A':
            self.player_ace += 1
        if self.player_total > 21 and self.player_ace > 0:
            self.player_total -= 10
            self.player_ace -= 1

    def dealer_draw(self):
        draw_card = np.random.choice(
            ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K'])
        self.dealer_cards.append(draw_card)
        self.dealer_total += self.CARD_VALUE[draw_card]
        if draw_card == 'A':
            self.dealer_ace += 1
        if self.dealer_total > 21 and self.dealer_ace > 0:
            self.dealer_total -= 10
            self.dealer_ace -= 1

    def result(self):
        reward = 0
        if self.player_total > 21:
            reward = -1
        elif self.dealer_total > 21:
            reward = 1
        elif self.player_total > self.dealer_total:
            reward = 1
        elif self.player_total < self.dealer_total:
            reward = -1
        else:
            reward = 0
        return reward

# 状態集合の定義
S_to_num = { }
state_num = 0
for player_score in range(12, 22):
    for ace_num in [ 0, 1 ]:
        for dealer_card in range(1, 11):
            S_to_num[ ( player_score, ace_num, dealer_card)] =  state_num
            state_num += 1
S_to_num["PlayerWin"] = state_num
S_to_num["PlayerLose"] = state_num + 1
S_to_num["Draw"] = state_num + 2

S = list(S_to_num.keys())
#print(S)

# 行動集合の定義
A_to_num = { "hit" : 0, "stand" : 1}
A = [ "hit" , "stand" ]

# 報酬関数の定義
def reward(s1, a, s2):
    if s2 =="PlayerWin":
        return 1
    elif s2 == "PlayerLose":
        return -1
    elif s2 == "Draw":
        return 0
    else:
        return 0
    
# 割引率の定義
gamma = 1.0

def generate_episode(pi, s_0, a_0):
    game = BlackJack(s_0=s_0)
    episode = [ [s_0, a_0] ]
    if a_0 == "hit":
        game.player_hit()
    elif a_0 == "stand":
        game.player_stand()
        
    s = game.state()
    
    episode[-1].append(reward(s_0, a_0, s))
    while not game.finish:
        a = np.random.choice(A, p=pi[S_to_num[s] , :])
        game.show_status()
        if a == "hit":
            game.player_hit()
        elif a == "stand":
            game.player_stand()
        next_s = game.state()
        r = reward(s, a, next_s)
        episode.append([ s, a, r ])
        s = next_s
    game.show_status()
    return episode

def Monte_Carlo_ES():
    Q = np.random.rand(  len(S), len(A) )
    pi = np.random.rand(  len(S), len(A) )
    for i in range(len(S)):
        pi[i] = pi[i, :] / np.sum(pi[i, :])
    Returns = [ [0]*len(A) for _ in range(len(S)) ]
    
    while True:
        s_0 = np.random.choice(S)
        # 終状態から実行しても意味がないので
        if s_0 == "PlayerWin" or s_0 == "PlayerLose" or s_0 == "Draw":
            continue
        a_0 = np.random.choice(A)
        # エピソード生成.
        episode = reversed(generate_episode(pi, s_0, a_0))
        new_Q = np.empty( (len(S), len(A)))
        
        R = 0
        for s_t, a_t, r_t in episode:
            s, a = S_to_num(s_t), A_to_num(a_t)
            R += r_t
            # 初回訪問
            if np.isnan(new_Q[s, a]):
                G = R
                Returns[s, a].append(G)
                Q[s, a] = Returns[s,a]     
Monte_Carlo_ES()

--------------------
Player (total : 21) [ 4 ]
Dealer (total : ??) [ 7, ?? ]
--------------------
--------------------
Player (total : 23) [ 4, 2 ]
Dealer (total : 18) [ 7 ]
--------------------
[[(17, 0, 1), 'hit', 0], [(21, 0, 7), 'hit', -1]]
